# Congestion Charges - Hard

You may need to create views to complete these questions - but you do not have permission to create tables or views in the default schema. Your SQL commands are executed by user scott in schema gisq - you may create or drop views and tables in schema scott but not in gisq.

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app18-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@20ca1207

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@76344eb6

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val camera = hiveCxt.table("sqlzoo.camera")
val keeper = hiveCxt.table("sqlzoo.keeper")
val vehicle = hiveCxt.table("sqlzoo.vehicle")
val image = hiveCxt.table("sqlzoo.image")
val permit = hiveCxt.table("sqlzoo.permit")

camera: DataFrame = [id: int, perim: string]
keeper: DataFrame = [id: int, name: string ... 1 more field]
vehicle: DataFrame = [id: string, keeper: int]
image: DataFrame = [camera: int, whn: string ... 1 more field]
permit: DataFrame = [reg: string, sdate: string ... 1 more field]

## 1.
When creating a view in scott you must specify the schema name of the sources and the destination.

## 2.
There are four types of permit. The most popular type means that this type has been issued the highest number of times. Find out the most popular type, together with the total number of permits issued.

In [5]:
(permit.groupBy("chargetype")
 .agg(count("reg").alias("cnt"))
 .orderBy(col("cnt").desc)
 .limit(1)
 .showHTML())

chargetype,cnt
Daily,27


## 3.
For each of the vehicles caught by camera 19 - show the registration, the earliest time at camera 19 and the time and camera at which it left the zone.

In [6]:
// registrations showing at camera 19, the earliest time
(image.filter(image("camera") === 19)
 .join(vehicle, (image("reg") === vehicle("id")))
 .groupBy("reg")
 .agg(min("whn").alias("earliest"))
 .join(image, "reg", joinType="left")
 .filter(col("earliest") < col("whn"))
 .groupBy("reg", "earliest")
 .agg(min("whn").alias("next"))
 .withColumnRenamed("reg", "reg1")
 .join(image, ((col("reg1") === image("reg")) && 
               (col("next") === image("whn"))))
 .select("reg", "earliest", "next", "camera")
 .showHTML())

reg,earliest,next,camera
SO 02 CSP,2007-02-25 07:51:10.0,2007-02-25 07:55:11.0,18
SO 02 DSP,2007-02-25 16:31:01.0,2007-02-25 17:42:41.0,19
SO 02 JSP,2007-02-25 17:14:11.0,2007-02-25 17:17:03.0,3
SO 02 TSP,2007-02-25 07:23:00.0,2007-02-25 07:26:31.0,19


## 4.
For all 19 cameras - show the position as IN, OUT or INTERNAL and the busiest hour for that camera.

In [7]:
(camera
 .withColumn("type", when(camera("perim").isNull, "INTERNAL")
             .otherwise(camera("perim")))
 .join(image.withColumn("hr", hour(image("whn"))), 
       (camera("id") === image("camera")))
 .groupBy("camera", "type", "hr")
 .agg(count("id").alias("n"))
 .groupBy("camera", "type", "hr")
 .max("n")
 .orderBy("camera")
 .showHTML())

camera,type,hr,max(n)
1,IN,6,1
2,IN,7,1
3,IN,18,2
3,IN,17,3
5,IN,7,1
8,IN,7,2
9,OUT,18,1
9,OUT,6,1
9,OUT,16,6
10,OUT,18,2


## 5.
Anomalous daily permits. Daily permits should not be issued for non-charging days. Find a way to represent charging days. Identify the anomalous daily permits.

In [8]:
(permit.filter((dayofweek(permit("sdate")).isin(List(1, 7): _*)) &&
               (permit("chargetype") === "Daily"))
 .showHTML())

reg,sdate,chargetype
SO 02 ATP,2007-01-21 00:00:00.0,Daily
SO 02 BTP,2007-02-03 00:00:00.0,Daily
SO 02 BTP,2007-02-04 00:00:00.0,Daily
SO 02 CTP,2007-01-21 00:00:00.0,Daily
SO 02 FTP,2007-02-25 00:00:00.0,Daily
SO 02 HTP,2006-01-21 00:00:00.0,Daily
SO 02 HTP,2006-01-22 00:00:00.0,Daily
SO 02 JTP,2007-01-21 00:00:00.0,Daily


## 6.
Issuing fines: Vehicles using the zone during the charge period, on charging days must be issued with fine notices unless they have a permit covering that day. List the name and address of such culprits, give the camera and the date and time of the first offence.

In [9]:
// vehicles with permits
val f = (vehicle.join(permit, (vehicle("id") === permit("reg")), joinType="left")
     .withColumn("sdate", to_timestamp(col("sdate")))
     .withColumn("edate", when(
        col("chargetype") === "Daily", col("sdate") + expr("interval 1 day")).when(
        col("chargetype") === "Weekly", col("sdate") + expr("interval 1 week")).when(
        col("chargetype") === "Monthly", col("sdate") + expr("interval 1 month")).when(
        col("chargetype") === "Annual", col("sdate") + expr("interval 1 year")))
     .join(image, "reg", joinType="right")
     .join(keeper.withColumnRenamed("id", "keeper"), "keeper")
     .filter((col("whn")<col("sdate")) || (col("whn")>col("edate"))))

val a = (f.groupBy("reg", "name", "address")
     .agg(min("whn").alias("first_offence")))

(a.alias("a")
 .join(f.alias("f").select("reg", "whn", "camera"), 
        ((a("reg") === f("reg")) && (a("first_offence") === f("whn"))))
 .select("a.reg", "name", "address", "first_offence", "camera")
 .showHTML())

reg,name,address,first_offence,camera
SO 02 GSP,"Incongruous, Ingrid",Irresolution Pl.,2007-02-25 07:10:00.0,5
SO 02 ASP,"Ambiguous, Arthur",Absorption Ave.,2007-02-25 06:10:13.0,1
SO 02 ISP,"Incongruous, Ingrid",Irresolution Pl.,2007-02-25 16:58:01.0,9
SO 02 JSP,"Inconspicuous, Iain",Interception Rd.,2007-02-25 17:07:00.0,3
SO 02 DSP,"Strenuous, Sam",Surjection Street,2007-02-25 16:29:11.0,18
SO 02 CSP,"Ambiguous, Arthur",Absorption Ave.,2007-02-25 06:57:31.0,17
SO 02 HSP,"Assiduous, Annie",Attribution Alley,2007-02-25 16:45:04.0,9


f: Dataset[Row] = [keeper: int, reg: string ... 8 more fields]
a: DataFrame = [reg: string, name: string ... 2 more fields]

In [10]:
spark.stop()